In [146]:
%load_ext autoreload
%autoreload 2
import numpy as np
import protograph as pt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
a=pt.ring_of_circulants_f2([1])
b=pt.ring_of_circulants_f2([-1])
(a*b).to_binary(3)

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [150]:
a=np.array([[[0,1]]])
a.shape

a=pt.ring_of_circulants_f2([1])
b=pt.ring_of_circulants_f2([-1])
a*b

pt.zeros(4)

protograph.array([[(),(),(),()],[(),(),(),()],[(),(),(),()],[(),(),(),()]])

In [149]:
a=pt.ring_of_circulants_f2([1])
b=pt.ring_of_circulants_f2([-1])
a*b

pt.zeros(4)

protograph.array([[(),(),(),()],[(),(),(),()],[(),(),(),()],[(),(),(),()]])

In [99]:
from bposd.css import css_code


def I(n):
    return pt.identity(n)

class lifted_hgp(css_code):

    def __init__(self,lift_parameter,a,b=None):

        '''
        Generates the lifted hypergraph product of the protographs a and b
        '''
        self.a=a

        self.a_m,self.a_n=self.a.shape

        if b is None:
            self.b=pt.copy(self.a)
        else:
            self.b=b
        
        self.b_m,self.b_n=self.b.shape

        self.hx1_proto=pt.kron(self.a,I(self.b_n))
        self.hx2_proto=pt.kron(I(self.a_m),self.b.T)
        self.hx_proto=pt.hstack([self.hx1_proto,self.hx2_proto])

        self.hz1_proto=pt.kron(I(self.a_n),self.b)
        self.hz2_proto=pt.kron(self.a.T,I(self.b_m))
        self.hz_proto=pt.hstack([self.hz1_proto,self.hz2_proto])

        super().__init__(self.hx_proto.to_binary(lift_parameter),self.hz_proto.to_binary(lift_parameter))

In [153]:
d=21

a=pt.array([[(0,1)]])
b=pt.array([[(0,d)]])

qcode=lifted_hgp(d*(d-1),a,b)

qcode.test()

<Unnamed CSS code>, (2,4)-[[840,2,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (2,4)-[[840,2,nan]]


True

In [154]:
def lpc1():
    proto_a=pt.array([
        [(0), (11), (7), (12)],
        [(1), (8), (1), (8)],
        [(11), (0), (4), (8)],
        [(6), (2), (4), (12)]
    ])
    return lifted_hgp(13,proto_a)

def p19a1():

    proto_a=pt.array([
        [(36), (), (), (), (), (0), (9)],
        [(9), (36), (), (), (), (), (0)],
        [(0), (9), (36), (), (), (), ()],
        [(), (0), (9), (36), (), (), ()],
        [(), (), (0), (9), (36), (), ()],
        [(), (), (), (0), (9), (36), ()],
        [(), (), (), (), (0), (9), (36)]
    ])

    proto_b=pt.array([[(0, 1, 6)]])

    return lifted_hgp(63,proto_b,proto_a)


qcode=lpc1()
qcode.canonical_logicals()
qcode.test()


hx=np.loadtxt("hx.txt").astype(int)
hz=np.loadtxt("hz.txt").astype(int)

assert np.array_equal(qcode.hx,qcode.hx)
assert np.array_equal(qcode.hz,qcode.hz)

<Unnamed CSS code>, (4,8)-[[416,18,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (4,8)-[[416,18,nan]]


In [155]:
a=pt.array([
        [(36), (), (), (), (), (0), (9)],
        [(9), (36), (), (), (), (), (0)],
        [(0), (9), (36), (), (), (), ()],
        [(), (0), (9), (36), (), (), ()],
        [(), (), (0), (9), (36), (), ()],
        [(), (), (), (0), (9), (36), ()],
        [(), (), (), (), (0), (9), (36)]
    ])


# a=pt.array([[(1,1),()],[(),(1,2)]])

# a=pt.array([[(),(0)]])
# a=pt.ring_of_circulants_f2(np.array([[(),(0)]]))
# a.coefficients
print(a.T)


[[(-36) (-9) (0) () () () ()]
 [() (-36) (-9) (0) () () ()]
 [() () (-36) (-9) (0) () ()]
 [() () () (-36) (-9) (0) ()]
 [() () () () (-36) (-9) (0)]
 [(0) () () () () (-36) (-9)]
 [(-9) (0) () () () () (-36)]]


/home/roffej92/github/bias_tailored_qec/protograph.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_proto=np.array(proto_array)


In [81]:
pt.ring_of_circulants_f2(list((2))).to_binary(3)

TypeError: 'int' object is not iterable

In [82]:
proto_a=pt.array([
        [(0), (11), (7), (12)],
        [(1), (8), (1), (8)],
        [(11), (0), (4), (8)],
        [(6), (2), (4), (12)]
    ])

print(proto_a)

[[(0) (11) (7) (12)]
 [(1) (8) (1) (8)]
 [(11) (0) (4) (8)]
 [(6) (2) (4) (12)]]


In [83]:
list((2,))

[2]

In [84]:
[(0,1)][0]

(0, 1)

In [85]:
[(0)]

[0]

In [177]:
a=[(0,1)]
try:
    a=list(a)
except TypeError:
    a=[a]

a=np.array(a).astype(int)
print(repr(a))
len(a.shape)

a.dtype=object
a

array([[0, 1]])


TypeError: Cannot change data-type for object array.

In [178]:
class test(np.ndarray):
    def __new__(cls,input_array):
        temp_proto=np.asarray(input_array)
        n=temp_proto.shape
        obj=np.empty(n).astype(pt.ring_of_circulants_f2)

        for i,ring_els in enumerate(temp_proto):
            obj[i]=pt.ring_of_circulants_f2(ring_els)
        return obj


a=[(0,1)]

test(a)

    


In [167]:
np.asarray({1,})

array({1}, dtype=object)